In [ ]:
import sqlite3
conn = sqlite3.connect('main.db') # insert user/password database file name here
cursor = conn.cursor() # create cursor object

In [ ]:
# def register(username:str, password:str, fName:str, lName:str) -> str:
def registerUser(username, password, fName, lName):
    cursor.execute(f"SELECT * FROM Users WHERE Username=?;", [username])
    if len(cursor.fetchall()) > 0 :
        return "Username already exists."
    else:
        cursor.execute("INSERT INTO Users VALUES (?, ?, ?, ?);", [username, password, fName, lName])
        cursor.execute("INSERT INTO Scores VALUES (?,?);", [username, 0])
        conn.commit()
        return "Successful"

In [ ]:
def loginUser(user, password):
    cursor.execute("SELECT * FROM Users WHERE Username = ?",[user]) # searches the database for the entered username
    result = cursor.fetchall()
    print(result)
    if len(result) == 0: # checks if an account with such a username exists
        return "No Username"
    correct_password = result[0][1] # assumes the correct password is kept in column index 1 of the database 
    if password == correct_password: # compares entered password and possible correct passwords (assuming duplicate usernames are allowed)
        return "Logged In"
    return "Incorrect Password"

In [ ]:
def correctness(questionNum, answer): #answer is entered as string or as 2-d array entry
    search_num = questionNum
    cursor.execute("SELECT * FROM Questions WHERE QuestionID = ?",[search_num])
    result = cursor.fetchall()
    if answer == result[0][4]:  # checks if entry equals the correct answer
        return result[0][3] # replace ? with the column index of the point value assigned to the question
    else:
        return 0

In [ ]:
import random
def shuffle(question_num):
    cursor.execute("SELECT * FROM Questions WHERE QuestionID = (?)",[question_num]) #parses database for question id
    result = cursor.fetchall()
    question_list = [result[0][4], result[0][5], result[0][6], result[0][7]] #assembles list of question options
    random.shuffle(question_list) #shuffles list
    return question_list #returns shuffled list

In [ ]:
def addPoints(questionId, answer, user): #note, as of 10/31, you can answer the same question multiple times and farm infinite points.
    newPoints = correctness(questionId, answer) #finds number of points earned from previous function
    cursor.execute("SELECT * FROM Scores WHERE Username = (?)",[user]) #finds number of points from the points table
    result = cursor.fetchall()
    updated_points = result[0][1] + newPoints #adds old points and newly earned points
    cursor.execute("UPDATE Scores SET Points = (?) WHERE Username = (?)", [updated_points, user]) #updates table value for username
    conn.commit()

In [ ]:
cursor.execute("SELECT * FROM Users;")
cursor.fetchall()

In [ ]:
def getContent(company):
    cursor.execute("SELECT * FROM Content where CompanyName=?", [company])
    result = cursor.fetchall()[0]
    return result[1][1:-1]

getContent("Apple")

In [ ]:
from werkzeug.wrappers import Request, Response
from flask import Flask, render_template, request, url_for, flash, redirect


app = Flask(__name__)
app.config['SECRET_KEY'] = 'superdupersecretkeythatnobodywouldeverguess'


@app.route("/", methods=('GET', 'POST'))
def index():
    return render_template('index.html')

@app.route("/register", methods=('GET', 'POST'))
def register():
    print("tried to register")
    if request.method == 'POST':
        username = str(request.form.get("uname"))
        password = str(request.form.get("psw"))
        firstname = str(request.form.get("fname"))
        lastname = str(request.form.get("lname"))
        out = registerUser(username, password, firstname, lastname)
        if out == "Successful":
            return redirect(url_for('index'))
        else:
            print("Register failed")
    
@app.route("/login", methods=('GET', 'POST'))
def login():
    print("tried to login")
    if request.method == 'POST':
        username = str(request.form.get("uname"))
        password = str(request.form.get("psw"))
        out = loginUser(username, password)
        if out == "Logged In":
            return redirect(url_for('home', user=username))
        else:
            print(out)
    
@app.route("/home")
def home():
    return render_template('home.html')

@app.route("/index", methods=('GET', 'POST'))
def index2():
    return render_template('index.html')

@app.route("/lessons", methods=["POST", "GET"])
def lessons():
    if request.method == "POST":
        # Handle form submission if needed
        pass
    return render_template('lessons.html')

@app.route("/practice", methods=["POST", "GET"])
def practice():
    if request.method == "POST":
        # Handle form submission if needed
        pass
    return render_template("practice.html")

@app.route("/lessonsfn", methods=["POST", "GET"])
def lessonsfn():
    if request.method == "POST":
        x = str(request.form.get("lesson_type"))
        print(x + "lessons")
        content = getContent(x)
        return render_template("lessoncontent.html", content=content)

@app.route("/practicefn", methods=["POST", "GET"])
def practicefn():
    if request.method == "POST":
        x = str(request.form.get("lesson_type"))
        print(x + "practice")
        return render_template("quiz.html", questions=questions)

@app.route("/output/<name>")
def output(name):
    
    #creates a new list that holds the information we were passed
    return_list = []
    return_list.append(name);
    
    # returns the scraped data as a list to output.html
    return render_template('output.html', output_list=return_list, output_string=name)

# this snippet of code is another syntactical inclusion - it's not as important to understand everything
# that's happening here, but it essentially decides where your flask app is going to run on the local device
if __name__ == '__main__':
    from werkzeug.serving import run_simple

    # the "8080" means that the flask app will run on http://localhost:8080/
    # you can change this to any other viable port if 8080 is currently in use
    # for example, you can change the line below to read "run_simple('localhost', 9000, app)", and then
    # access the flask app through http://localhost:9000/ - the port is completely changeable and
    # does not affect your code at all beyond where you access it from
    run_simple('localhost', 8080, app)